In [6]:
import pandas as pd
from recipe_scrapers import scrape_me
import parse_ingredient
import re
import numpy as np

In [2]:
scraper = scrape_me('https://www.allrecipes.com/recipe/158968/spinach-and-feta-turkey-burgers/')

In [3]:
ingredients = scraper.ingredients()

In [18]:
def parse_ingredients(ingredients):
    data = []
    for ingredient in ingredients:
        print(ingredient)
        info = (ingredient, np.NaN, np.NaN, np.NaN)
        try:
            info = parse_ingredient.parse(ingredient)
            info = info.as_dict()
        except Exception as e:
            pass
            data.append(info)
            continue
        
        #puts original ingredient if low confidence
        if info["confidence"] <= .05:
            data.append((ingredient, np.NaN, np.NaN, np.NaN))
            continue
        
        #puts all information together
        data.append((info["product"], info["quantity"], info["unit"], info["usda_info"]["category"]))
        
    return pd.DataFrame(data, columns=["product", "quantity", "unit", "category"])


In [24]:
shopping_list = parse_ingredients(ingredients)

2 eggs, beaten
None
{'ingredientRaw': '2 eggs, beaten', 'confidence': 0.9764371000000001, 'ingredientParsed': {'product': 'eggs', 'usdaInfo': {'category': 'Dairy and Egg Products', 'matchMethod': 'exact', 'description': 'Egg, whole, raw, fresh', 'fdcId': '171287'}, 'preparationNotes': 'beaten', 'productSizeModifier': None, 'unit': None, 'quantity': 2.0}, 'error': None}
2 cloves garlic, minced
None
{'ingredientRaw': '2 cloves garlic, minced', 'confidence': 0.9817117, 'ingredientParsed': {'product': 'garlic', 'usdaInfo': {'category': None, 'matchMethod': 'exact', 'description': 'Garlic, raw', 'fdcId': '787793'}, 'preparationNotes': 'minced', 'productSizeModifier': None, 'unit': 'clove', 'quantity': 2.0}, 'error': None}
4 ounces feta cheese
None
{'ingredientRaw': '4 ounces feta cheese', 'confidence': 0.9916780000000001, 'ingredientParsed': {'product': 'feta cheese', 'usdaInfo': {'category': 'Dairy and Egg Products', 'matchMethod': 'exact', 'description': 'Cheese, feta', 'fdcId': '173420'}

In [25]:
shopping_list

,product,quantity,unit,category
0,eggs,2.0,None,Dairy and Egg Products
1,garlic,2.0,clove,None
2,feta cheese,4.0,ounce,Dairy and Egg Products
3,"1 (10 ounce) box frozen chopped spinach, thawe...",NaN,NaN,NaN
4,turkey,2.0,pound,Poultry Products


Note: I can join all of the different types of units (value counts) together when I group by product